# Experimental Code

<i>Version 1.0</i>

## Dependencies

The package, [bitsets](https://bitsets.readthedocs.io/en/stable/), provides a memory-efficient pure-python immutable ordered set data type for working with large numbers of subsets from a predetermined pool of objects.

In [1]:
from bitsets import bitset

Algebra's are defined in JSON format.

In [2]:
import json

## Allen's Interval Algebra using Bitsets

In [24]:
class Algebra(object):

    def __init__(self, filename):
        """An algebra is created from a JSON file containing the algebra's
        relation and transitivity table definitions.
        """
        with open(filename, 'r') as f:
            self.algebra_dict = json.load(f)

        self.name = self.algebra_dict["Name"]
        self.description = self.algebra_dict["Description"]
        self.rel_info = self.algebra_dict["Relations"]
        self.AlgBitSet = bitset(self.name, tuple(self.rel_info.keys()))  # A class object
        self.identity = self.AlgBitSet.supremum

        # Setup the transitivity table used by Relation Set multiplication
        self.transitivity_table = dict()
        tabledefs = self.algebra_dict["TransTable"]
        for rel1 in tabledefs:
            self.transitivity_table[rel1] = dict()
            for rel2 in tabledefs[rel1]:
                self.transitivity_table[rel1][rel2] = self.AlgBitSet(tuple(tabledefs[rel1][rel2]))
                
    def __str__(self):
        return f"<{self.name}: {self.description}>"
    
    def converse(self, rel_or_relset):
        '''Return the converse of a relation (str) or relation set (bitset).'''
        if isinstance(rel_or_relset, str):
            return self.rel_info[rel_or_relset]["Converse"]
        else:
            return self.AlgBitSet((self.converse(r) for r in rel_or_relset.members()))
        
    def relset(self, elements):
        return self.AlgBitSet(elements)
        
    def relname(self, rel):
        return self.rel_info[rel]["Name"]
    
    def reltran(self, rel):
        return self.rel_info[rel]["Transitive"]
    
    def add(self, relset1, relset2):
        '''Addition for relation sets is equivalent to set intersection.'''
        return relset1.intersection(relset2)
    
    def mult(self, relset1, relset2):
        '''Multiplication is done, element-by-element, on the cross-product
        of the two sets using the algebra's transitivity table, and
        then reducing those results to a single relation set using set
        union.
        '''
        result = self.AlgBitSet.infimum  # the empty relation set
        for r1 in relset1:
            for r2 in relset2:
                result = result.union(self.transitivity_table[r1][r2])
        return result
    
    def check_multiplication_identity(self, verbose=False):
        """Check the validity of the multiplicative identity for every
        combination of singleton relset.  :param verbose: Print out
        the details of each test :return: True or False

        """
        count = 0
        result = True
        rels = self.identity
        for r in rels:
            r_rs = self.relset((r,))
            for s in rels:
                count += 1
                s_rs = self.relset((s,))
                prod1 = self.mult(r_rs, s_rs)
                prod2 = self.converse(self.mult(self.converse(s_rs), self.converse(r_rs)))
                if prod1 != prod2:
                    if verbose:
                        print("FAIL:")
                        print(f"      r    = {r_rs}")
                        print(f"      s    = {s_rs}")
                        print(f"( r *  s)  = {prod1}")
                        print(f"(si * ri)i = {prod2}")
                        print(f"{prod1} != {prod2}")
                    result = False
        if verbose:
            print(f"\n{self.name} -- Multiplication Identity Check:")
        if result:
            if verbose:
                print(f"PASSED . {count} products tested.")
        else:
            if verbose:
                print("FAILED. See FAILURE output above.")
        return result

    def print_info(self):
        mapping = {frozenset([u'ProperInterval', u'Point']): "Int",
                   frozenset([u'Point']): "Pt",
                   frozenset([u'ProperInterval']): "PInt",
                   frozenset([u'Region']): "Region"
                   }
        print(f"  Algebra Name: {self.name}")
        print(f"   Description: {self.description}")
#        print(f" Equality Rels: {self.equality_relations}")
        print("     Relations:")
        print("{:>25s} {:>25s} {:>10s} {:>10s} {:>10s} {:>7s} {:>7s}".format("NAME (ABBREV)", "CONVERSE (ABBREV)",
                                                                             "REFLEXIVE", "SYMMETRIC", "TRANSITIVE",
                                                                             "DOMAIN", "RANGE"))
        # TODO: Vary spacing between columns based on max word lengths
#        for r in sorted_rels:
#            print("{:>19s} ({:>3s}) {:>19s} ({:>3s}) {:>8s} {:>10s} {:>10s} {:>8s} {:>7s}".format(r.long_name,
#                                                                                                  r.short_name,
#                                                                                                  r.converse.long_name,
#                                                                                                  r.converse.short_name,
#                                                                                                  str(r.is_reflexive),
#                                                                                                  str(r.is_symmetric),
#                                                                                                  str(r.is_transitive),
#                                                                                                  mapping[r.domain],
#                                                                                                  mapping[r.range]))


In [25]:
allen = Algebra("IntervalAlgebra.json")
print(allen)

<LinearTimeIntervalAlgebra: Allen's algebra of proper time intervals>


In [26]:
allen.converse(allen.relset(('B','M','OI')))

LinearTimeIntervalAlgebra(['BI', 'MI', 'O'])

In [27]:
allen.converse('B')

'BI'

In [28]:
allen.identity

LinearTimeIntervalAlgebra(['B', 'BI', 'D', 'DI', 'E', 'F', 'FI', 'M', 'MI', 'O', 'OI', 'S', 'SI'])

In [29]:
allen.relname('B')

'Before'

In [30]:
allen.transitivity_table

{'B': {'B': LinearTimeIntervalAlgebra(['B']),
  'BI': LinearTimeIntervalAlgebra(['B', 'BI', 'D', 'DI', 'E', 'F', 'FI', 'M', 'MI', 'O', 'OI', 'S', 'SI']),
  'D': LinearTimeIntervalAlgebra(['B', 'D', 'M', 'O', 'S']),
  'DI': LinearTimeIntervalAlgebra(['B']),
  'E': LinearTimeIntervalAlgebra(['B']),
  'F': LinearTimeIntervalAlgebra(['B', 'D', 'M', 'O', 'S']),
  'FI': LinearTimeIntervalAlgebra(['B']),
  'M': LinearTimeIntervalAlgebra(['B']),
  'MI': LinearTimeIntervalAlgebra(['B', 'D', 'M', 'O', 'S']),
  'O': LinearTimeIntervalAlgebra(['B']),
  'OI': LinearTimeIntervalAlgebra(['B', 'D', 'M', 'O', 'S']),
  'S': LinearTimeIntervalAlgebra(['B']),
  'SI': LinearTimeIntervalAlgebra(['B'])},
 'BI': {'B': LinearTimeIntervalAlgebra(['B', 'BI', 'D', 'DI', 'E', 'F', 'FI', 'M', 'MI', 'O', 'OI', 'S', 'SI']),
  'BI': LinearTimeIntervalAlgebra(['BI']),
  'D': LinearTimeIntervalAlgebra(['BI', 'D', 'F', 'MI', 'OI']),
  'DI': LinearTimeIntervalAlgebra(['BI']),
  'E': LinearTimeIntervalAlgebra(['BI']),
  'F

In [31]:
before = allen.relset(['B'])
during = allen.relset(['D'])

bxd = allen.mult(before, during)
bxd

LinearTimeIntervalAlgebra(['B', 'D', 'M', 'O', 'S'])

In [32]:
foobar = allen.relset(['D','M','F','SI'])
print(foobar.members())
allen.add(bxd,foobar)

('D', 'F', 'M', 'SI')


LinearTimeIntervalAlgebra(['D', 'M'])

In [33]:
allen.check_multiplication_identity()

True

In [34]:
allen.print_info()

  Algebra Name: LinearTimeIntervalAlgebra
   Description: Allen's algebra of proper time intervals
     Relations:
            NAME (ABBREV)         CONVERSE (ABBREV)  REFLEXIVE  SYMMETRIC TRANSITIVE  DOMAIN   RANGE
